In [7]:
import numpy as np
import time
import os

import matplotlib.pyplot as plt
import pandas as pd
import random

import json

import tensorflow.keras
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import models
import shap

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_ENABLE_GPU_GARBAGE_COLLECTION"] = 'false'
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], False)

DEBUG = False

In [8]:
column_names = ['timestamp','open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target']
file = '/home/joren/Coding/cryptodata/Normalized_labelled/AAVEUSDT.bin'
modelfile = 'models/model_small_3'
frame_size = 240

In [9]:
# load data
field_info = [
    { "type": np.uint64, "count": 1 },
    { "type": np.double, "count": 17 },
    { "type": np.int64, "count": 1 }
]
BYTES_EIGHT = 8

def read_bin_full_file(file):
    f = open(file, 'rb')
    b = f.read(-1)

    BYTES_TO_READ = 0
    for field in field_info:
        BYTES_TO_READ += BYTES_EIGHT * field["count"]

    data = []
    BYTES_READ = 0
    for i in range(0, int(os.path.getsize(file) / BYTES_TO_READ)):
        row = []

        for idx, field in enumerate(field_info):
            row += np.frombuffer(b, dtype=field["type"], count=field["count"], offset=BYTES_READ).tolist()

            BYTES_READ += BYTES_EIGHT * field["count"]

        data.append(row)
    return np.array(data, dtype=np.float16)

# train-test split
# train-test split
def df_split(df):  
    X = df.drop(columns=['timestamp','target'], axis=0).to_numpy()
    Y = df['target'].to_numpy()

    X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, Y, test_size=0.25, shuffle=False)

    y_train_raw = to_categorical(y_train_raw, 3).tolist()
    y_test_raw = to_categorical(y_test_raw, 3).tolist()

    X_train = []
    y_train = []
    # y_train_weights = []
    for i in range(frame_size, X_train_raw.shape[0]): #frame size up to size of array
        X_train.append(X_train_raw[i-frame_size:i])
        # y_train.append(y_train_raw[i-frame_size:i]) # dit wil ik dus graag veranderen naar y_train_raw[i] zodat we enkel op het einde de output hebben
        y_train.append(y_train_raw[i])
    X_train, y_train = np.array(X_train), np.array(y_train)
    # X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))


    X_test = []
    y_test = []
    for i in range(frame_size, X_test_raw.shape[0]): #frame size up to size of array
        X_test.append(X_test_raw[i-frame_size:i])
        # y_test.append(y_test_raw[i-frame_size:i])
        y_test.append(y_test_raw[i])
    X_test, y_test = np.array(X_test), np.array(y_test)

    if DEBUG:
        print(f"""
        X_train shape: {X_train.shape}
        y_train shape: {y_train.shape}
        """)

        print(f"""
        X_train[0] shape: {X_train[0].shape}
        y_train[0] shape: {y_train[0].shape}
        """)

        print(f"X_train[0]: {X_train[0]}")
        print(f"y_train[0]: {y_train[0]}")

    return X_train, X_test, y_train, y_test

In [10]:
# load model
model = models.load_model(modelfile)

In [11]:
## prepare data for prediction
data = read_bin_full_file(file)
df2 = pd.DataFrame(data, columns=column_names)
df2.fillna(0, inplace=True)
startloc = round(len(df2)*0.95)
df = df2.iloc[-500:]
del(df2)

In [12]:
X_train, X_test, y_train, y_test = df_split(df)

NameError: name 'to_categorical' is not defined

In [ ]:
explainer = shap.DeepExplainer(model, X_train)

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.


In [ ]:
shap_values = explainer.shap_values(X_test)

`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


AttributeError: in user code:

    File "/home/joren/.local/lib/python3.9/site-packages/shap/explainers/_deep/deep_tf.py", line 243, in grad_graph  *
        out = self.model(shap_rAnD)
    File "/home/joren/.local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/joren/.local/lib/python3.9/site-packages/shap/explainers/_deep/deep_tf.py", line 26, in custom_record_gradient
        out = tf_backprop._record_gradient("shap_"+op_name, inputs, attrs, results)

    AttributeError: Exception encountered when calling layer "lstm" (type LSTM).
    
    module 'tensorflow.python.eager.backprop' has no attribute '_record_gradient'
    
    Call arguments received:
      • inputs=tf.Tensor(shape=(520, 240, 17), dtype=float32)
      • mask=None
      • training=False
      • initial_state=None
